In [1]:
from glob import glob

from collections import defaultdict
import json
import os

In [2]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai5-aiservices1787699516.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [10]:
questions = defaultdict(set)
q = []
for f in glob('base-questions-v5*/*'):
    b = os.path.split(f)[0]
    with open(f) as fopen:
        data = json.load(fopen)
    data = '\n'.join(data)
    data = data.split('\n\n')
    if len(data) == 1:
        data = data[0].split('\n')
    else:
        datas = []
        for d in data:
            splitted = d.split('\n')
            splitted = [s.strip() for s in splitted]
            datas.extend(splitted)
        data = datas
    data = ['.'.join(d.split('.')[1:]).strip() for d in data]
    data = [d for d in data if len(d) > 3]
        
    data = set(data)
    questions[b] |= data

In [11]:
total_questions = 0
all_questions = []
for k, v in questions.items():
    total_questions += len(v)
    all_questions.extend(v)
    
total_questions

6363

In [12]:
all_questions = sorted(all_questions)

In [13]:
all_questions

['"Adakah diterima dalam Islam untuk seorang Muslim di Malaysia memberi zakat fitrah melalui platform digital? Apakah syarat-syarat yang perlu dipatuhi?"',
 '"Adakah diterima zakat dari pendapatan yang diperolehi melalui industri hiburan yang tidak sepenuhnya selari dengan prinsip Islam di Malaysia?"',
 '"Adakah hukum fiqah membenarkan wanita Muslim di Malaysia untuk menjadi hakim di mahkamah syariah?"',
 '"Adakah memelihara kucing dengan tujuan komersial diterima dalam fiqah Islam di Malaysia?"',
 '"Adakah sah solat Jumaat yang dilaksanakan dalam kampus universiti yang memiliki pelbagai mazhab pelajar di Malaysia dari aspek fiqah?"',
 '"Adakah sah solat Jumaat yang dilakukan secara maya melalui siaran langsung dalam situasi pandemi, bagi mereka yang tidak dapat hadir ke masjid di Malaysia?"',
 '"Adakah sah zakat fitrah diberikan melalui platform pembayaran digital di Malaysia mengikut hukum fiqah Islam?"',
 '"Adakah urus niaga jual beli mata wang asing (forex) secara online diterima d

In [8]:
!mkdir answer-question-chatgpt4-v5
# !rm -rf answer-question-chatgpt4-v3/*

mkdir: cannot create directory ‘answer-question-chatgpt4-v5’: File exists


In [9]:
from tqdm import tqdm

for i in tqdm(range(len(all_questions))):
    filename = f'answer-question-chatgpt4-v5/{i}.json'
    if os.path.exists(filename):
        continue
        
    try:
        message_text = [
            {"role":"user","content": f"{all_questions[i]}, jawab dalam bahasa melayu"},
        ]
        completion = openai.ChatCompletion.create(
          engine="gpt-4",
          messages = message_text,
          temperature=1.0,
          max_tokens=1024,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        splitted = completion.choices[0]['message']['content'].split('\n')
    except:
        splitted = None
    with open(filename, 'w') as fopen:
        json.dump(splitted, fopen)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 6363/6363 [21:06:22<00:00, 11.94s/it]


In [19]:
qa, texts = [], []
for i in range(len(all_questions)):
    q = all_questions[i]
    
    filename = f'answer-question-chatgpt4-v5/{i}.json'
    if not os.path.exists(filename):
        continue
        
    with open(filename) as fopen:
        answer = json.load(fopen)
    
    if answer is None:
        continue
        
    answer = '\n'.join(answer)
    
    qa.append({
        'question': q.strip(),
        'answer': answer.strip()
    })
    
    texts.extend([q.strip(), answer.strip()])
    
len(qa)

6363

In [24]:
len(texts)

12726

In [21]:
with open('answer-question-chatgpt4-v5.texts', 'w') as fopen:
    for t in set(texts):
        fopen.write(f'{json.dumps(t)}\n')

In [22]:
!cp answer-question-chatgpt4-v5.texts ../ctranslate2

In [25]:
mapping = {}
with open('../ctranslate2/answer-question-chatgpt4-v5.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
len(mapping)

12726

In [29]:
with open('malaysian-general-qa-v5.jsonl', 'w') as fopen:
    for q in qa:
        q['question_ms'] = mapping.get(q['question'])
        q['answer_ms'] = mapping.get(q['answer'])
        fopen.write(f'{json.dumps(q)}\n')

In [30]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='malaysian-general-qa-v5.jsonl',
    path_in_repo='malaysian-general-qa-v5.jsonl',
    repo_id='mesolitica/chatgpt4-malaysian-general-qa',
    repo_type='dataset',
)

'https://huggingface.co/datasets/mesolitica/chatgpt4-malaysian-general-qa/blob/main/malaysian-general-qa-v5.jsonl'